In [31]:
import pandas as pd
import numpy as np
from multiprocessing.pool import ThreadPool
from threading import Thread

In [2]:
# Downloading the data
data = {}

URLS = [("2018", "https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20221017-145753/valeursfoncieres-2018.txt"),
        ("2019", "https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20221017-150616/valeursfoncieres-2019.txt"),
        ("2020", "https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20221017-151136/valeursfoncieres-2020.txt"),
        ("2021", "https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20221017-151704/valeursfoncieres-2021.txt"),
        ("Q12022", "https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20221017-152027/valeursfoncieres-2022-s1.txt")]

In [3]:
# Async download data for a quicker result.
def download_data(year: str, url: str):
    return year, pd.read_csv(url, sep='|')
    
with ThreadPool(len(URLS)) as pool:
    results = pool.starmap_async(download_data, URLS)
    for result in results.get():
        year = result[0]
        dataframe = result[1]
        data[year] = dataframe

/var/folders/fj/8pmdh2z120lbtvdmmc2407th0000gn/T/ipykernel_76863/2757997852.py:3: DtypeWarning: Columns (18,23,24,26,28,31,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  return year, pd.read_csv(url, sep='|')
/var/folders/fj/8pmdh2z120lbtvdmmc2407th0000gn/T/ipykernel_76863/2757997852.py:3: DtypeWarning: Columns (18,23,24,26,28,31,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  return year, pd.read_csv(url, sep='|')
/var/folders/fj/8pmdh2z120lbtvdmmc2407th0000gn/T/ipykernel_76863/2757997852.py:3: DtypeWarning: Columns (14,18,23,24,26,28,29,31,32,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  return year, pd.read_csv(url, sep='|')
/var/folders/fj/8pmdh2z120lbtvdmmc2407th0000gn/T/ipykernel_76863/2757997852.py:3: DtypeWarning: Columns (14,18,23,24,26,28,29,31,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  return year, pd.read_csv(url, sep='|')
/var/

In [4]:
# Check that all dataframes have the same structure.
columns = [set(df.columns) for df in data.values()]

from itertools import groupby

def all_equal(iterable):
    g = groupby(iterable)
    return next(g, True) and not next(g, False)

all_equal(columns)

True

In [5]:
# Create one dataframe:
dataframes = list(data.values())
df = dataframes[0]
for next_dataframe in dataframes[1:]:
    df = pd.concat([df, next_dataframe])
display(df)

,Code service sages,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,...,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,03/01/2018,Vente,...,NaN,1,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,03/01/2018,Vente,...,NaN,2,2.0,Appartement,NaN,73.0,4.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,04/01/2018,Vente,...,NaN,0,1.0,Maison,NaN,163.0,4.0,S,NaN,949.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,04/01/2018,Vente,...,NaN,0,1.0,Maison,NaN,51.0,2.0,AG,JARD,420.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,04/01/2018,Vente,...,NaN,0,1.0,Maison,NaN,163.0,4.0,AG,JARD,420.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1429088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,21/06/2022,Vente,...,NaN,2,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN
1429089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,21/06/2022,Vente,...,NaN,1,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN
1429090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,30/06/2022,Vente,...,NaN,1,2.0,Appartement,NaN,15.0,1.0,NaN,NaN,NaN
1429091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,10/02/2022,Adjudication,...,NaN,1,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN


In [6]:
# Export to .parquet to have a smaller filesize and avoid repeated downloading.
df.to_csv("downloaded_data.csv")
del df

/var/folders/fj/8pmdh2z120lbtvdmmc2407th0000gn/T/ipykernel_76863/1275080020.py:5: DtypeWarning: Columns (15,19,24,25,27,29,30,31,32,33,34,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("downloaded_data.csv")


In [87]:
# Read the data and use a sample to study it, free the memory after.
df = pd.read_csv("downloaded_data.csv")
sample = df.sample(1_000_000).copy()
del df

/var/folders/fj/8pmdh2z120lbtvdmmc2407th0000gn/T/ipykernel_66595/645220393.py:2: DtypeWarning: Columns (15,19,24,25,27,29,30,31,32,33,34,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("downloaded_data.csv")


In [88]:
# Explore the data
display(sample.columns)
display(sample.info())

Index(['Unnamed: 0', 'Code service sages', 'Reference document',
       '1 Articles CGI', '2 Articles CGI', '3 Articles CGI', '4 Articles CGI',
       '5 Articles CGI', 'No disposition', 'Date mutation', 'Nature mutation',
       'Valeur fonciere', 'No voie', 'B/T/Q', 'Type de voie', 'Code voie',
       'Voie', 'Code postal', 'Commune', 'Code departement', 'Code commune',
       'Prefixe de section', 'Section', 'No plan', 'No Volume', '1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot',
       'Nombre de lots', 'Code type local', 'Type local', 'Identifiant local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain'],
      dtype='object')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 3501462 to 4543916
Data columns (total 44 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   Unnamed: 0                  1000000 non-null  int64  
 1   Code service sages          0 non-null        float64
 2   Reference document          0 non-null        float64
 3   1 Articles CGI              0 non-null        float64
 4   2 Articles CGI              0 non-null        float64
 5   3 Articles CGI              0 non-null        float64
 6   4 Articles CGI              0 non-null        float64
 7   5 Articles CGI              0 non-null        float64
 8   No disposition              1000000 non-null  int64  
 9   Date mutation               1000000 non-null  object 
 10  Nature mutation             1000000 non-null  object 
 11  Valeur fonciere             989319 non-null   object 
 12  No voie                     611357 non-null   floa

None

In [89]:
# Check for empty columns or columns with lots of Nans. Don't drop Data above the lot and lot size because we will reformat them later.
def drop_empty_cols(df):
    len_df = len(df)
    for col in df:
        if "lot" in col:
            continue
        if df[col].isna().sum() / len_df > 0.5:
            print(f"{col} have too much missing data. Dropping it.")
            df = df.drop(col, axis=1)
    return df

sample = drop_empty_cols(sample)
display(sample.isna().sum())

Code service sages have too much missing data. Dropping it.
Reference document have too much missing data. Dropping it.
1 Articles CGI have too much missing data. Dropping it.
2 Articles CGI have too much missing data. Dropping it.
3 Articles CGI have too much missing data. Dropping it.
4 Articles CGI have too much missing data. Dropping it.
5 Articles CGI have too much missing data. Dropping it.
B/T/Q have too much missing data. Dropping it.
Prefixe de section have too much missing data. Dropping it.
No Volume have too much missing data. Dropping it.
Identifiant local have too much missing data. Dropping it.
Nature culture speciale have too much missing data. Dropping it.


Unnamed: 0                         0
No disposition                     0
Date mutation                      0
Nature mutation                    0
Valeur fonciere                10681
No voie                       388643
Type de voie                  407085
Code voie                       8869
Voie                            8887
Code postal                     8910
Commune                            0
Code departement                   0
Code commune                       0
Section                           39
No plan                            0
1er lot                       684821
Surface Carrez du 1er lot     912204
2eme lot                      922885
Surface Carrez du 2eme lot    975611
3eme lot                      986670
Surface Carrez du 3eme lot    997432
4eme lot                      995574
Surface Carrez du 4eme lot    999334
5eme lot                      997975
Surface Carrez du 5eme lot    999734
Nombre de lots                     0
Code type local               430579
T

In [90]:
# Dropping the columns that don't carry useful information.
def drop_useless_columns(df):
    to_drop = ["Unnamed: 0", "Nature mutation", "No voie", "Voie", "Commune", "No disposition", "Code type local"]
    return df.drop(to_drop, axis=1)

sample = drop_useless_columns(sample)
display(sample)

,Date mutation,Valeur fonciere,Type de voie,Code voie,Code postal,Code departement,Code commune,Section,No plan,1er lot,...,4eme lot,Surface Carrez du 4eme lot,5eme lot,Surface Carrez du 5eme lot,Nombre de lots,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Surface terrain
3501462,03/04/2019,"1265000,00",BD,0376,6210.0,6,79,BR,396,NaN,...,NaN,NaN,NaN,NaN,0,Maison,205.0,7.0,S,500.0
14739727,31/12/2021,"1950802,60",RUE,0018,97250.0,972,25,C,493,NaN,...,NaN,NaN,NaN,NaN,0,Appartement,45.0,1.0,J,2000.0
9144049,20/11/2020,"335000,00",RUE,0485,62136.0,62,502,CN,36,NaN,...,NaN,NaN,NaN,NaN,0,Maison,219.0,7.0,S,755.0
6110660,01/03/2019,"179000,00",RUE,3460,77500.0,77,108,AR,545,281.0,...,NaN,NaN,NaN,NaN,1,Dépendance,0.0,0.0,NaN,NaN
4471476,04/03/2019,"150000,00",RUE,0253,33770.0,33,498,AS,216,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,AG,769.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1348119,12/03/2018,"70000,00",NaN,B054,39210.0,39,317,ZI,47,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,T,1732.0
10330507,28/02/2020,NaN,NaN,B007,95340.0,95,487,ZA,40,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,T,1245.0
13586791,12/07/2021,"50000,00",PL,0030,72610.0,72,355,A,107,NaN,...,NaN,NaN,NaN,NaN,0,Dépendance,0.0,0.0,S,580.0
1163089,20/12/2018,"780000,00",NaN,B046,34480.0,34,44,G,363,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,T,3100.0


In [91]:
# Checking % of missing values.
sample.isna().sum() / len(sample)

Date mutation                 0.000000
Valeur fonciere               0.010681
Type de voie                  0.407085
Code voie                     0.008869
Code postal                   0.008910
Code departement              0.000000
Code commune                  0.000000
Section                       0.000039
No plan                       0.000000
1er lot                       0.684821
Surface Carrez du 1er lot     0.912204
2eme lot                      0.922885
Surface Carrez du 2eme lot    0.975611
3eme lot                      0.986670
Surface Carrez du 3eme lot    0.997432
4eme lot                      0.995574
Surface Carrez du 4eme lot    0.999334
5eme lot                      0.997975
Surface Carrez du 5eme lot    0.999734
Nombre de lots                0.000000
Type local                    0.430579
Surface reelle bati           0.431238
Nombre pieces principales     0.431238
Nature culture                0.319633
Surface terrain               0.319633
dtype: float64

In [92]:
# Drop the columns that have too much missing values or can't be treated.
sample = sample.drop(["Type de voie", "Surface Carrez du 5eme lot", "Type local", "Surface reelle bati", 
                      "Nombre pieces principales", "Nature culture", "Surface terrain"], axis=1)
display(sample)

,Date mutation,Valeur fonciere,Code voie,Code postal,Code departement,Code commune,Section,No plan,1er lot,Surface Carrez du 1er lot,2eme lot,Surface Carrez du 2eme lot,3eme lot,Surface Carrez du 3eme lot,4eme lot,Surface Carrez du 4eme lot,5eme lot,Nombre de lots
3501462,03/04/2019,"1265000,00",0376,6210.0,6,79,BR,396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
14739727,31/12/2021,"1950802,60",0018,97250.0,972,25,C,493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9144049,20/11/2020,"335000,00",0485,62136.0,62,502,CN,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
6110660,01/03/2019,"179000,00",3460,77500.0,77,108,AR,545,281.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4471476,04/03/2019,"150000,00",0253,33770.0,33,498,AS,216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1348119,12/03/2018,"70000,00",B054,39210.0,39,317,ZI,47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
10330507,28/02/2020,NaN,B007,95340.0,95,487,ZA,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
13586791,12/07/2021,"50000,00",0030,72610.0,72,355,A,107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1163089,20/12/2018,"780000,00",B046,34480.0,34,44,G,363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [93]:
# Convert non numeric fields to float
def convert_to_float(value: str):
    if pd.isna(value):
        return
    value = str(value)
    try:
        return float(value.replace(',', '.'))
    except ValueError:
        return 

sample[["1er lot", "Surface Carrez du 1er lot", 
       "2eme lot", "Surface Carrez du 2eme lot",
       "3eme lot", "Surface Carrez du 3eme lot",
       "4eme lot", "Surface Carrez du 4eme lot",
       "5eme lot", "Valeur fonciere"]] = sample[["1er lot", "Surface Carrez du 1er lot", 
       "2eme lot", "Surface Carrez du 2eme lot",
       "3eme lot", "Surface Carrez du 3eme lot",
       "4eme lot", "Surface Carrez du 4eme lot",
       "5eme lot", "Valeur fonciere"]].applymap(convert_to_float)

# Convert missing values to 0 fort Surface fields.
sample[["1er lot", "Surface Carrez du 1er lot", 
       "2eme lot", "Surface Carrez du 2eme lot",
       "3eme lot", "Surface Carrez du 3eme lot",
       "4eme lot", "Surface Carrez du 4eme lot",
       "5eme lot"]] = sample[["1er lot", "Surface Carrez du 1er lot", 
       "2eme lot", "Surface Carrez du 2eme lot",
       "3eme lot", "Surface Carrez du 3eme lot",
       "4eme lot", "Surface Carrez du 4eme lot",
       "5eme lot"]].fillna(0)

# Drop the remaining Nan values:
display(sample)

,Date mutation,Valeur fonciere,Code voie,Code postal,Code departement,Code commune,Section,No plan,1er lot,Surface Carrez du 1er lot,2eme lot,Surface Carrez du 2eme lot,3eme lot,Surface Carrez du 3eme lot,4eme lot,Surface Carrez du 4eme lot,5eme lot,Nombre de lots
3501462,03/04/2019,1265000.0,0376,6210.0,6,79,BR,396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
14739727,31/12/2021,1950802.6,0018,97250.0,972,25,C,493,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
9144049,20/11/2020,335000.0,0485,62136.0,62,502,CN,36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
6110660,01/03/2019,179000.0,3460,77500.0,77,108,AR,545,281.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4471476,04/03/2019,150000.0,0253,33770.0,33,498,AS,216,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1348119,12/03/2018,70000.0,B054,39210.0,39,317,ZI,47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
10330507,28/02/2020,NaN,B007,95340.0,95,487,ZA,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
13586791,12/07/2021,50000.0,0030,72610.0,72,355,A,107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1163089,20/12/2018,780000.0,B046,34480.0,34,44,G,363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [94]:
# Check for missing values again:
sample.isna().sum() / len(sample)

Date mutation                 0.000000
Valeur fonciere               0.010681
Code voie                     0.008869
Code postal                   0.008910
Code departement              0.000000
Code commune                  0.000000
Section                       0.000039
No plan                       0.000000
1er lot                       0.000000
Surface Carrez du 1er lot     0.000000
2eme lot                      0.000000
Surface Carrez du 2eme lot    0.000000
3eme lot                      0.000000
Surface Carrez du 3eme lot    0.000000
4eme lot                      0.000000
Surface Carrez du 4eme lot    0.000000
5eme lot                      0.000000
Nombre de lots                0.000000
dtype: float64

In [95]:
# Drop missing values, we still have many datapoints.
sample = sample.dropna()
display(sample)

,Date mutation,Valeur fonciere,Code voie,Code postal,Code departement,Code commune,Section,No plan,1er lot,Surface Carrez du 1er lot,2eme lot,Surface Carrez du 2eme lot,3eme lot,Surface Carrez du 3eme lot,4eme lot,Surface Carrez du 4eme lot,5eme lot,Nombre de lots
3501462,03/04/2019,1265000.0,0376,6210.0,6,79,BR,396,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
14739727,31/12/2021,1950802.6,0018,97250.0,972,25,C,493,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
9144049,20/11/2020,335000.0,0485,62136.0,62,502,CN,36,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
6110660,01/03/2019,179000.0,3460,77500.0,77,108,AR,545,281.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4471476,04/03/2019,150000.0,0253,33770.0,33,498,AS,216,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14840995,25/02/2021,270000.0,4614,75018.0,75,118,BW,117,22.0,24.02,60.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1348119,12/03/2018,70000.0,B054,39210.0,39,317,ZI,47,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
13586791,12/07/2021,50000.0,0030,72610.0,72,355,A,107,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1163089,20/12/2018,780000.0,B046,34480.0,34,44,G,363,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [96]:
# Export to csv for data exploration:
sample.to_csv("cleaned_data.csv")